# A2

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from collections import OrderedDict
from tqdm.notebook import tqdm

## Setting up data

In [ ]:
dataset_dbi = ImageFolder('/content/drive/MyDrive/420_data/dbi_subset')
dataset_sdd = ImageFolder('/content/drive/MyDrive/420_data/sdd_subset')

if(dataset_dbi.classes != dataset_sdd.classes):
  raise Exception("Not the same classes in each dataset")

print(f'DBI Dataset \nDatabase length: {len(dataset_dbi)} \nDatabase classes: {dataset_dbi.classes}')
print(f'\nSDD Datast \nDatabase length: {len(dataset_sdd)} \nDatabase classes: {dataset_sdd.classes}')

In [ ]:
random_seed = 45
torch.manual_seed(random_seed);

In [ ]:
test_pct = 0.3
val_pct = 0.1

# DBI data split
test_dbi_size = int(len(dataset_dbi)*test_pct)
dataset_dbi_size = len(dataset_dbi) - test_dbi_size

val_dbi_size = int(dataset_dbi_size*val_pct)
train_dbi_size = dataset_dbi_size - val_dbi_size

train_ds_dbi, val_ds_dbi, test_ds_dbi = random_split(dataset_dbi, [train_dbi_size, val_dbi_size, test_dbi_size])

# SDD data split
test_sdd_size = int(len(dataset_sdd)*0.8)
dataset_sdd_size = len(dataset_sdd) - test_sdd_size

val_sdd_size = int(dataset_sdd_size*val_pct)
train_sdd_size = dataset_sdd_size - val_sdd_size

train_ds_sdd, val_ds_sdd, test_ds_sdd = random_split(dataset_sdd, [train_sdd_size, val_sdd_size, test_sdd_size])




In [ ]:
img, label = train_ds_dbi[6]
# print(dataset_dbi.classes[label])
# plt.imshow(img)
# print(type(img))

In [ ]:
class DogBreedDataset(Dataset):
    
    def __init__(self, ds, transform=None):
        self.ds = ds
        self.transform = transform
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        img, label = self.ds[idx]
        if self.transform:
            img = self.transform(img)  
            return img, label

### Transforming images

In [ ]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_transform = transforms.Compose([
#    transforms.Resize((224, 224)),
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224, padding=4, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
#    transforms.Normalize(*imagenet_stats, inplace=True)
    
])


val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
#    transforms.Normalize(*imagenet_stats, inplace=True)
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(),
#    transforms.Normalize(*imagenet_stats, inplace=True)
])

In [ ]:
# DBI
train_dataset_dbi = DogBreedDataset(train_ds_dbi, train_transform)
val_dataset_dbi = DogBreedDataset(val_ds_dbi, val_transform)
test_dataset_dbi = DogBreedDataset(test_ds_dbi, test_transform)

# SDD
train_dataset_sdd = DogBreedDataset(train_ds_sdd, train_transform)
val_dataset_sdd = DogBreedDataset(val_ds_sdd, val_transform)
test_dataset_sdd = DogBreedDataset(test_ds_sdd, test_transform)

# Full Data SDD
full_data_sdd = DogBreedDataset(dataset_sdd, test_transform)

In [ ]:
batch_size =64

# DBI Dataloaders
train_dl_dbi = DataLoader(train_dataset_dbi, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl_dbi = DataLoader(val_dataset_dbi, batch_size*2, num_workers=2, pin_memory=True)
test_dl_dbi = DataLoader(test_dataset_dbi, batch_size*2, num_workers=2, pin_memory=True)

# SDD Dataloader
test_dl_sdd = DataLoader(full_data_sdd, batch_size, num_workers=2, pin_memory=True)


In [ ]:
def show_batch(dl):
    for img, lb in dl:
        fig, ax = plt.subplots(figsize=(16, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(img.cpu(), nrow=16).permute(1,2,0))
        break

# show_batch(dataset_dl_sdd)

## Network

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class ImageClassificationBase(nn.Module):
    # training step
    def training_step(self, batch):
        img, targets = batch
        out = self(img)
        loss = F.nll_loss(out, targets)
        acc = accuracy(out, targets) 
        return {'train_acc':acc.detach(), 'train_loss':loss}
    
    # validation step
    def validation_step(self, batch):
        img, targets = batch
        out = self(img)
        loss = F.nll_loss(out, targets)
        acc = accuracy(out, targets)
        return {'val_acc':acc.detach(), 'val_loss':loss.detach()}
    
    # validation epoch end
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss':epoch_loss.item(), 'val_acc':epoch_acc.item()}
        
    # print result end epoch
    def epoch_end(self, epoch, result):
        print("Epoch [{}] : train_loss: {:.4f}, train_acc: {:.4f}, test_acc: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result["train_loss"], result["train_acc"], result["test_acc"], result["val_loss"], result["val_acc"]))
        

#### CNN Model From Assignment

In [ ]:
class DogBreedClassificationCNN(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        
        self.network = nn.Sequential(
            
            # Layer 1
            nn.Conv2d(3, 16, 3, stride=1, padding=1),   # 224 * 244 * 16
            nn.BatchNorm2d(16),
            nn.ReLU(),

            # Layer 2
            nn.Conv2d(16, 16, 3, stride=1, padding=1),  # 224 * 224 * 16    
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                         # 112 * 112 * 16
            
            # Layer 3
            nn.Conv2d(16, 8, 3, stride=1, padding=1),   # 112 * 112 * 8
            nn.BatchNorm2d(8),
            nn.ReLU(), 

            # Layer 4
            nn.Conv2d(8, 8, 3, stride=1, padding=1),    # 112 * 112 * 8
            nn.ReLU(),
            nn.MaxPool2d(2,2),                          # 56 * 56 * 8
            # nn.Dropout(0.5),

            # Fully connected layyer
            nn.Flatten(),
            nn.Linear(56 * 56 * 8, 32),
            nn.ReLU(),
            # nn.Dropout(0.5),
            nn.Linear(32, 7),

            nn.LogSoftmax(dim = 1)
        )
    
    def forward(self, xb):
        return self.network(xb)

#### ResNet-18

In [ ]:
class DogBreedResnet18(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        
        self.network = models.resnet18(pretrained=False)

        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Sequential(
            nn.Linear(num_ftrs, 7),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, xb):
        return self.network(xb)

#### ResNet18 Pretrained

In [ ]:
class DogBreedPretrainedResnet18(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        
        self.network = models.resnet18(pretrained=True)

        # Freeze layers, except last layer for finetuning
        for param in self.network.parameters():
            param.requires_grad = False

        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Sequential(
            nn.Linear(num_ftrs, 7),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, xb):
        return self.network(xb)

#### ResNet34 Pretrained

In [ ]:
class DogBreedPretrainedResnet34(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        
        self.network = models.resnet34(pretrained=True)

        # Freeze layers, except last layer for finetuning
        for param in self.network.parameters():
            param.requires_grad = False
            
        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Sequential(
            nn.Linear(num_ftrs, 7),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, xb):
        return self.network(xb)

#### ResNeXt32

In [ ]:
class DogBreedPretrainedResnext32(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        
        self.network = models.resnext101_32x8d(pretrained=True)

        # Freeze layers, except last layer for finetuning
        for param in self.network.parameters():
            param.requires_grad = False

        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Sequential(
            nn.Linear(num_ftrs, 7),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, xb):
        return self.network(xb)

Moving Data to GPU

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(d, device) for d in data]
    else:
        return data.to(device, non_blocking=True)

In [ ]:
class DeviceDataLoader:
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __len__(self):
        return len(self.dl)
    
    def __iter__(self):
        for batch in self.dl:
            yield to_device(batch, self.device)

In [ ]:
# getting default device
device = get_default_device()
print(device)

# moving train dataloader and val dataloader to gpue
train_dl_dbi = DeviceDataLoader(train_dl_dbi, device)
val_dl_dbi = DeviceDataLoader(val_dl_dbi, device)
test_dl_dbi = DeviceDataLoader(test_dl_dbi, device)

test_dl_sdd = DeviceDataLoader(test_dl_sdd, device)

## CNN & Resnet18 Training on the DBI

In [ ]:
cnn_model = DogBreedClassificationCNN()
to_device(cnn_model, device);

In [ ]:
resnet18_model = DogBreedResnet18()
to_device(resnet18_model, device);

In [ ]:
pretrained_resnet18 = DogBreedPretrainedResnet18()
to_device(pretrained_resnet18, device);

In [ ]:
pretrained_resnet34 = DogBreedPretrainedResnet34()
to_device(pretrained_resnet34, device);

In [ ]:
pretrained_resnext32 = DogBreedPretrainedResnext32()
to_device(pretrained_resnext32, device);

In [ ]:
# check the model 
def try_batch(model, dl):
    for imgs, labels in dl:
        print("images shape : ", imgs.shape)
        print("labels : ", labels)
        outs = model(imgs)                                  # Change model object here
        print("outs.shape :", outs.shape)
        print("outs : ", outs)
        break
        
# try_batch(cnn_model, train_dl_dbi)

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
        

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, test_loader, weight_decay=0, grad_clip=None, opt_func = torch.optim.Adam):
    torch.cuda.empty_cache()
    history = []
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # set up one cycle lr scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        
        # Training phase
        model.train()       
        train_losses = []
        train_accs = []
        lrs = []
        for batch in tqdm(train_loader):
            output = model.training_step(batch)

            train_accs.append(output['train_acc'])
            train_losses.append(output['train_loss'].detach())
            
            # calculates gradients
            output['train_loss'].backward()
            
            # check gradient clipping 
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
                
            # perform gradient descent and modifies the weights
            optimizer.step()
            
            # reset the gradients
            optimizer.zero_grad()
            
            # record and update lr
            lrs.append(get_lr(optimizer))
            
            # modifies the lr value
            sched.step()
            
        # Validation phase
        result = evaluate(model, val_loader)
        result['test_acc'] = evaluate(model, test_loader)['val_acc']
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['train_acc'] = torch.stack(train_accs).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
        
      
    return history
        
    

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [ ]:
model = pretrained_resnext32
result = evaluate(model, test_dl_sdd)["val_acc"]
result

In [ ]:
# set hyperparams
num_epochs = 5
opt_func = torch.optim.SGD

max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4

In [ ]:
history = fit_one_cycle(num_epochs, max_lr, model, train_dl_dbi, val_dl_dbi, test_dl_dbi, weight_decay, grad_clip, opt_func)

In [ ]:
num_epochs = 3
max_lr = 0.001
history += fit_one_cycle(num_epochs, max_lr, model, train_dl_dbi, val_dl_dbi, test_dl_dbi, weight_decay, grad_clip, opt_func)

In [ ]:
num_epochs = 2
max_lr = 0.0001
history += fit_one_cycle(num_epochs, max_lr, model, train_dl_dbi, val_dl_dbi, test_dl_dbi, weight_decay, grad_clip, opt_func)

### Plotting Over 10 Epochs

In [ ]:
test_accs_data = []
train_acc_data = []
validation_acc_data = []
epoch_num = list(range(len(history)))

for epoch in history:
  test_accs_data.append(epoch['test_acc'])
  train_acc_data.append(epoch['train_acc'])
  validation_acc_data.append(epoch['val_acc'])

plt.plot(epoch_num, test_accs_data, c='green', label='test_accuracy', marker='x')
plt.plot(epoch_num, train_acc_data, c='blue', label='train_accuraxy', marker='x')
plt.plot(epoch_num, validation_acc_data, c='red', label='val_accuraxy', marker='x')
plt.xlabel('epochs')
plt.ylabel('accuracy')
leg = plt.legend(loc='upper center')
plt.show()

#### Running trained model on SDD dataset

In [ ]:
result = evaluate(model, test_dl_sdd)["val_acc"]
result

## Dataset Classification of SDD vs DBI

In [ ]:
dataset = ImageFolder('/content/drive/MyDrive/420_data/sdd_dbi_subset')

In [ ]:
test_pct = 0.3
test_size = int(len(dataset)*test_pct)
dataset_size = len(dataset) - test_size

val_pct = 0.1
val_size = int(dataset_size*val_pct)
train_size = dataset_size - val_size


train_size, val_size, test_size

In [ ]:
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])
len(train_ds), len(val_ds), len(test_ds)

In [ ]:
img, label = train_ds[6]
# print(dataset.classes[label])
# plt.imshow(img)
# print(type(img))

In [ ]:
train_dataset = DogBreedDataset(train_ds, train_transform)
val_dataset = DogBreedDataset(val_ds, val_transform)
test_dataset = DogBreedDataset(test_ds, test_transform)

In [ ]:
batch_size =64

# Create DataLoaders
train_dl = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_dataset, batch_size*2, num_workers=2, pin_memory=True)
test_dl = DataLoader(test_dataset, batch_size*2, num_workers=2, pin_memory=True)

In [ ]:
# getting default device
device = get_default_device()
print(device)

# moving train dataloader and val dataloader to gpu
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
test_dl = DeviceDataLoader(test_dl, device)

In [ ]:
class DogBreedPretrainedResnext32(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        
        self.network = models.resnext101_32x8d(pretrained=True)

        # Freeze layers, except last layer for finetuning
        for param in self.network.parameters():
            param.requires_grad = False

        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Sequential(
            nn.Linear(num_ftrs, 2),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, xb):
        return self.network(xb)



In [ ]:
model = DogBreedPretrainedResnext32()
to_device(model, device);
# result = evaluate(model, test_dl)["val_acc"]
# result

In [ ]:
# set hyperparams
num_epochs = 5
opt_func = torch.optim.SGD

max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
history = fit_one_cycle(num_epochs, max_lr, model, train_dl, val_dl, test_dl, weight_decay, grad_clip, opt_func)

In [ ]:
num_epochs = 3
max_lr = 0.001
history += fit_one_cycle(num_epochs, max_lr, model, train_dl, val_dl, test_dl, weight_decay, grad_clip, opt_func)

In [ ]:
num_epochs = 2
max_lr = 0.0001
history += fit_one_cycle(num_epochs, max_lr, model, train_dl, val_dl, test_dl, weight_decay, grad_clip, opt_func)

In [ ]:
test_accs_data = []
train_acc_data = []
val_acc_data = []
epoch_num = list(range(len(history)))

for epoch in history:
  test_accs_data.append(epoch['test_acc'])
  train_acc_data.append(epoch['train_acc'])
  val_acc_data.append(epoch['val_acc'])

plt.plot(epoch_num, test_accs_data, c='green', label='test_accuracy', marker='x')
plt.plot(epoch_num, train_acc_data, c='blue', label='train_accuraxy', marker='x')
plt.plot(epoch_num, val_acc_data, c='red', label='val_accuracy', marker='x')
plt.xlabel('epochs')
plt.ylabel('accuracy')
leg = plt.legend(loc='upper center')
plt.show()